In [2]:
!pip install ultralytics
!pip install pywhatkit
!pip install opencv-python
!pip install mediapipe
print("---------------------------------------------------")
print("DONE")

---------------------------------------------------
DONE


In [12]:
# Import necessary libraries
from __future__ import print_function
import os
import smtplib
import cv2
import numpy as np
import time
import threading
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText 
from ultralytics import YOLO 
import pywhatkit as kit  
from email import encoders
from email.mime.base import MIMEBase
import time
import smtplib
import threading
import mediapipe as mp
model = YOLO("yolov11n.pt") 
print("---------------------------------------------------")
print("DONE")

---------------------------------------------------
DONE


In [13]:
EMAIL_ADDRESS = "" 
EMAIL_PASSWORD = "" 

EMAIL_COOLDOWN = 300 
lastWhatsAppTime = 0  
WHATSAPP_COOLDOWN = 30 
VIDEO_DURATION = 15

PHONE_NUMBER = ""  

IMAGE_FOLDER = "images"  
VIDEO_FOLDER = "videos" 

MESSAGE_BODY = "A stranger was discovered. Please check the image."
message = "A stranger was discovered. Please check the image."

os.makedirs(IMAGE_FOLDER, exist_ok=True)
os.makedirs(VIDEO_FOLDER, exist_ok=True)

print("---------------------------------------------------")
print("DONE")

---------------------------------------------------
DONE


In [14]:
def sendEmail(image, videoName, num_persons_detected):
    server = smtplib.SMTP('smtp.gmail.com', 587)
    msg = __constructEmail(image, videoName, num_persons_detected)
    
    try:
        server.ehlo()  
        server.starttls() 
        server.ehlo()  
        
        server.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
        server.sendmail(EMAIL_ADDRESS, EMAIL_ADDRESS, msg.as_string())
        print("Email sent successfully!") 

    except Exception as e:
        print(f'An error occurred while sending email: {e}')
    
    finally:
        server.quit()
print("---------------------------------------------------")
print("DONE")

---------------------------------------------------
DONE


In [18]:
def __constructEmail(image, videoName, num_persons_detected):
    msg = MIMEMultipart()
    msg['Subject'] = "Warning: Danger Detected!"
    msg['From'] = EMAIL_ADDRESS
    msg['To'] = EMAIL_ADDRESS

    current_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

    body = f"""
    <html>
    <body>
        <h1 style="color: red; font-weight: bold;">Alert: Intruder detected!</h1>
        <p style="color: #555; font-weight: bold;">A stranger was discovered.</p>
        <p style="color: #555; font-weight: bold;">Please check the attached video for more details.</p>
        <br><hr><br>
        <p style="color: #555; font-weight: bold;">Number of persons detected: <span style="color: red;"><strong>{num_persons_detected}</strong></span></p>
        <p style="color: #555; font-weight: bold;">Detection Time: <span style="color: red;"><strong>{current_time}</strong></span></p>
    
        <img src="cid:image" alt="Detected Image" style="width:400px;"/><br>
    </body>
    </html>

    """

    msg.attach(MIMEText(body, 'html')) 

    imageName = 'image_' + str(time.time()) + '.png'
    imagePath = os.path.join(IMAGE_FOLDER, imageName)
    cv2.imwrite(imagePath, image)
    print(f"Image saved: {imagePath}")

    with open(imagePath, 'rb') as fp:
        img = MIMEImage(fp.read())
        img.add_header('Content-ID', '<image>')  
        msg.attach(img)

    videoPath = os.path.join(VIDEO_FOLDER, videoName)
    if os.path.exists(videoPath):
        with open(videoPath, 'rb') as fp:
            video = MIMEBase('application', 'octet-stream')
            video.set_payload(fp.read())
            encoders.encode_base64(video)
            video.add_header('Content-Disposition', f'attachment; filename="{videoName}"')
            msg.attach(video)
        print(f"Video attached: {videoPath}")
    else:
        print(f"Video not found: {videoPath}")

    return msg
print("---------------------------------------------------")
print("DONE")

---------------------------------------------------
DONE


In [19]:
def sendWhatsApp(image, videoName):
    global lastWhatsAppTime  

    currentTime = time.time()  

    if currentTime - lastWhatsAppTime >= WHATSAPP_COOLDOWN:
        imageName = 'image_' + str(currentTime) + '.png'
        imagePath = os.path.join(IMAGE_FOLDER, imageName)
        cv2.imwrite(imagePath, image)
        print(f"WhatsApp image saved: {imagePath}")

        try:
            kit.sendwhats_image(PHONE_NUMBER, imagePath, "Anomaly detected. Please check the image.")
            print("WhatsApp image sent successfully.")

            lastWhatsAppTime = currentTime  # Update the last WhatsApp sent time
        except Exception as e:
            print(f'An error while sending WhatsApp message: {e}')

        videoPath = os.path.join(VIDEO_FOLDER, videoName)
        if os.path.exists(videoPath):
            try:
                kit.sendwhats_image(PHONE_NUMBER, videoPath, "Here is the detected video.")
                print("WhatsApp video sent successfully.")
            except Exception as e:
                print(f'An error while sending WhatsApp video: {e}')
        else:
            print(f"WhatsApp video not found: {videoPath}")
    else:
        print("WhatsApp cooldown active. Message not sent.")


print("---------------------------------------------------")
print("DONE")

---------------------------------------------------
DONE


In [20]:
mp_face = mp.solutions.face_detection
mp_pose = mp.solutions.pose

face_detection = mp_face.FaceDetection(min_detection_confidence=0.5)
pose_detection = mp_pose.Pose(min_detection_confidence=0.5)

# Video capture settings
wc_feed = cv2.VideoCapture(0)

isReading = True
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = None
lastEmailTime = 0
lastVideoStartTime = 0
isSendingFrames = False
videoName = ""

os.makedirs(VIDEO_FOLDER, exist_ok=True)
os.makedirs(IMAGE_FOLDER, exist_ok=True)

def slow_down_video(video_path, slow_factor):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    new_video_path = video_path.replace('.avi', '_slow.avi')
    out = cv2.VideoWriter(new_video_path, fourcc, fps / slow_factor, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        out.write(frame)

        time.sleep(1 / (fps / slow_factor))

    cap.release()
    out.release()
    print(f"Slowed video saved: {new_video_path}")

while isReading:
    ret, feed = wc_feed.read()
    if not ret:
        break

    results = model(feed)

    detections = results[0].boxes.data.numpy() 
    currentTime = time.time()

    face_found = False
    pose_found = False
    num_persons_detected = 0 

    face_results = face_detection.process(cv2.cvtColor(feed, cv2.COLOR_BGR2RGB))
    if face_results.detections:
        face_found = True 

    pose_results = pose_detection.process(cv2.cvtColor(feed, cv2.COLOR_BGR2RGB))
    if pose_results.pose_landmarks:
        pose_found = True  

    for detection in detections:
        x1, y1, x2, y2, conf, cls = detection
        if conf > 0.5: 
            if int(cls) == 0: 
                num_persons_detected += 1  
                
                cv2.rectangle(feed, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
                cv2.putText(feed, "7aramy...-_-", (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.25, (0, 0, 255), 2)

                if face_found and pose_found and currentTime - lastEmailTime > EMAIL_COOLDOWN:
                    lastEmailTime = currentTime
                    imageName = 'image_' + str(time.time()) + '.png'
                    imagePath = os.path.join(IMAGE_FOLDER, imageName)
                    cv2.imwrite(imagePath, feed)
                    print(f"Image saved: {imagePath}")
                
                    videoName = 'video_' + str(time.time()) + '.avi'
                    videoPath = os.path.join(VIDEO_FOLDER, videoName)
                    out = cv2.VideoWriter(videoPath, fourcc, 20.0, (640, 480))
                
                    if not out.isOpened():
                        print("Error: Failed to open video writer.")
                    else:
                        print(f"Started recording video: {videoPath}")
                        lastVideoStartTime = currentTime
                        isSendingFrames = True  # Start recording frames

    if isSendingFrames:
        if currentTime - lastVideoStartTime < VIDEO_DURATION:
            out.write(feed)
        else:
            out.release()
            out = None
            isSendingFrames = False
            print(f"Video saved: {videoPath}")

            slow_down_video(videoPath, slow_factor=3)  

            thread_email = threading.Thread(target=sendEmail, args=(feed, videoName, num_persons_detected))
            thread_email.start()

            thread_whatsapp = threading.Thread(target=sendWhatsApp, args=(feed, videoName))
            thread_whatsapp.start()

    cv2.imshow("Detection", feed)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
wc_feed.release()
if out is not None:
    out.release()
cv2.destroyAllWindows()
print("---------------------------------------------------")
print("DONE")


0: 480x640 1 person, 165.3ms
Speed: 6.0ms preprocess, 165.3ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
Image saved: images\image_1729547668.0396917.png
Started recording video: videos\video_1729547668.061692.avi

0: 480x640 1 person, 178.7ms
Speed: 42.0ms preprocess, 178.7ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 137.0ms
Speed: 4.0ms preprocess, 137.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 149.0ms
Speed: 3.0ms preprocess, 149.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 117.0ms
Speed: 3.0ms preprocess, 117.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 115.0ms
Speed: 3.0ms preprocess, 115.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 110.0ms
Speed: 3.0ms preprocess, 110.0ms inference, 1.0ms postprocess per image at shape (1, 3, 48